# **Introducción a los Servicios Fundamentales de AWS e implementación de seguridad**

# Introducción

En este notebook, exploraremos cuatro servicios fundamentales de AWS:

1. **IAM (Identity and Access Management)** 🛡️
2. **AWS Networking: VPC, Subnets, NAT, Internet Gateway, Route Tables, ACL** 🌐
3. **Máquinas Virtuales (EC2)** 💻
4. **Contenedores (ECS)** 📦
5. **Computación Serverless (Lambda)** ⚡

Veremos las características de cada uno y cómo se implementan en AWS mediante este ejercicio.

## Diagrama de ejercicio general

![Diagram](assets/diagram.png)

## Explicación Arquitectura en AWS 

En este ejercicio vamos a crear una arquitectura en AWS que utiliza una VPC con subnets públicas y privadas:

- **Subnet pública**: alojará una aplicación web. 🌐
- **Subnet privada**: contendrá una base de datos. 💾

## Componentes clave:

- **Internet Gateway**: permitirá el acceso externo. 🌍
- **NAT Gateway**: permitirá que la base de datos acceda a internet sin ser expuesta. 🔒
- **Security Groups**: protegerán los recursos controlando el tráfico. 🛡️
- **ECR (Elastic Container Registry)**: almacenará imágenes de contenedores utilizadas por la aplicación. 📦
- **Lambda**: se empleará para hacer llamadas al servicio web. ⚡
- **S3**: almacenará los logs de las llamadas Lambda. La Lambda tendrá un rol con acceso exclusivo a un bucket S3. 🗃️
- **Session Manager**: se usará para una conexión segura a las instancias. 🔐
- **Tablas de enrutamiento**: gestionarán el flujo del tráfico dentro de la VPC. 🛣️

Esta arquitectura asegura una comunicación eficiente y segura entre los diferentes componentes.


# IAM (Identity and Access Management)

**IAM** es un servicio web que ayuda a controlar de forma segura el acceso a los recursos de AWS. 🛡️

Características:
- Gestión centralizada de usuarios y accesos 👥
- Autenticación y autorización granular 🔐
- Integración con servicios AWS 🔗
- Soporte para identidades federadas 🌍
- Sin costo adicional 💸


## Conceptos Clave

- **Cuenta Root**: Al crear una cuenta de AWS, se crea una cuenta root por defecto. Esta cuenta tiene acceso completo a todos los recursos y servicios de AWS y **no debe ser compartida**. 🔑
- **Usuarios**: Son entidades que creas en AWS para representar personas o servicios que interactúan con los recursos de AWS. Cada usuario tiene un conjunto único de credenciales. 👤
- **Grupos**: Son colecciones de usuarios. No pueden contener otros grupos, solo usuarios. Los usuarios pueden pertenecer a uno o más grupos. 👥
- **Roles**: Son similares a los usuarios, pero están destinados a ser asumidos por cualquier entidad que necesite acceso temporal a recursos específicos. 🎭
- **Políticas**: Definen permisos y pueden ser adjuntadas a usuarios, grupos o roles. 📜


### Ejemplo: Asignar nuevo usuario grupo con acceso a S3

En este ejercicio, vamos a crear un usuario y un grupo en AWS. El grupo tendrá permisos exclusivamente para enlistar y crear buckets en S3.


In [2]:
from IPython.display import IFrame

pdf_path = 'assets/iam.pdf'

IFrame(pdf_path, width=1000, height=500)


### Ejemplo: Crear un Rol de Administrador y Asumirlo por el Usuario
En este ejercicio, crearemos un rol de tipo Administrador, que tendrá permisos para crear y leer cualquier recurso en la cuenta de AWS. Además, otorgaremos al usuario los permisos necesarios para asumir este rol y ejecutar dichas acciones

In [1]:
from IPython.display import IFrame

pdf_path = 'assets/roleassume.pdf'

IFrame(pdf_path, width=1000, height=500)

### Documentación

Consultar esta información para más detalles:
- [Crear usuarios en IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_users_create.html)
- [Crear grupos en IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_groups_create.html)
- [Crear roles para servicios en IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_roles_create_for-service.html)
- [Crear roles en IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_roles_create.html)
- [Roles de IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_roles.html)
- [Asumir roles de AWS](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_roles_use.html)

# Setup Inicial para ejecutar proyecto 🚀

### Importar bibliotecas 🚀

In [3]:
import boto3
import json

### Configurar el cliente de AWS (asegúrate de tener tus credenciales configuradas) 🚀

In [4]:
iam = boto3.client('iam')
ec2 = boto3.client('ec2')
lambda_client = boto3.client('lambda')
ecs = boto3.client('ecs')

# AWS Networking: VPC, Subnets, NAT, Internet Gateway, Route Tables, ACL 🌐

AWS Networking se centra en la configuración y gestión de redes dentro de AWS utilizando varios componentes clave:

- **VPC (Virtual Private Cloud):** Una red virtual aislada dentro de AWS donde se despliegan recursos. 🏠
- **Subnets:** Segmentos de una VPC que permiten organizar y asegurar los recursos de manera granular. 📊
- **NAT (Network Address Translation):** Permite que instancias en subnets privadas accedan a Internet sin exponer sus direcciones IP privadas. 🔄
- **Internet Gateway:** Un componente que permite la comunicación entre instancias en una VPC y la Internet. 🌍
- **Route Tables:** Definen las reglas de enrutamiento para controlar el tráfico dentro de la VPC y hacia fuera. 🛣️
- **ACL (Access Control Lists):** Listas de control de acceso que proporcionan una capa de seguridad adicional a nivel de subred, permitiendo o denegando tráfico. 🔒

Estos componentes juntos permiten diseñar y gestionar redes seguras y eficientes en la nube de AWS.


## Creación y Configuración de una VPC 🏗️

Una Virtual Private Cloud (VPC) es una red virtual en la nube de AWS, aislada lógicamente de otras redes virtuales. A continuación, se detalla cómo crear y configurar una VPC usando Boto3, la biblioteca de AWS para Python.

In [12]:
import boto3

# Crea una VPC
response = ec2.create_vpc(
    CidrBlock='10.0.0.0/16'
)

vpc_id = response['Vpc']['VpcId']
print(f'VPC creada con ID: {vpc_id}')

# Habilita el DNS hostname para la VPC
ec2.modify_vpc_attribute(
    VpcId=vpc_id,
    EnableDnsSupport={'Value': True}
)

ec2.modify_vpc_attribute(
    VpcId=vpc_id,
    EnableDnsHostnames={'Value': True}
)

# Crea una etiqueta para la VPC
ec2.create_tags(
    Resources=[vpc_id],
    Tags=[{'Key': 'Name', 'Value': 'DemoVPC'}]
)


VPC creada con ID: vpc-054eacb87342c9240


{'ResponseMetadata': {'RequestId': '151f16ce-a9ce-4ffa-964a-b5e9353d4c8a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '151f16ce-a9ce-4ffa-964a-b5e9353d4c8a',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '221',
   'date': 'Tue, 06 Aug 2024 14:18:51 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

## Agregar Subnets


In [13]:
# Crear la subnet pública
response_public_subnet = ec2.create_subnet(
    VpcId=vpc_id,
    CidrBlock='10.0.1.0/24',
    AvailabilityZone='us-east-1a'
)

public_subnet_id = response_public_subnet['Subnet']['SubnetId']
print(f'Subnet pública creada con ID: {public_subnet_id}')

# Crear una etiqueta para la subnet pública
ec2.create_tags(
    Resources=[public_subnet_id],
    Tags=[{'Key': 'Name', 'Value': 'PublicSubnet'}]
)

# Crear la subnet privada
response_private_subnet = ec2.create_subnet(
    VpcId=vpc_id,
    CidrBlock='10.0.2.0/24',
    AvailabilityZone='us-east-1a'
)

private_subnet_id = response_private_subnet['Subnet']['SubnetId']
print(f'Subnet privada creada con ID: {private_subnet_id}')

# Crear una etiqueta para la subnet privada
ec2.create_tags(
    Resources=[private_subnet_id],
    Tags=[{'Key': 'Name', 'Value': 'PrivateSubnet'}]
)

# Crear un Internet Gateway
response_igw = ec2.create_internet_gateway()
igw_id = response_igw['InternetGateway']['InternetGatewayId']
print(f'Internet Gateway creado con ID: {igw_id}')

# Adjuntar el Internet Gateway a la VPC
ec2.attach_internet_gateway(
    InternetGatewayId=igw_id,
    VpcId=vpc_id
)

# Crear una etiqueta para el Internet Gateway
ec2.create_tags(
    Resources=[igw_id],
    Tags=[{'Key': 'Name', 'Value': 'DemoIGW'}]
)

# Obtener la tabla de rutas predeterminada
response_route_tables = ec2.describe_route_tables(
    Filters=[
        {
            'Name': 'vpc-id',
            'Values': [vpc_id]
        }
    ]
)

route_table_id = response_route_tables['RouteTables'][0]['RouteTableId']
print(f'Tabla de rutas predeterminada con ID: {route_table_id}')

# Crear una ruta en la tabla de rutas predeterminada que apunte al Internet Gateway
ec2.create_route(
    RouteTableId=route_table_id,
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=igw_id
)

# Asociar la tabla de rutas predeterminada con la subnet pública
ec2.associate_route_table(
    RouteTableId=route_table_id,
    SubnetId=public_subnet_id
)

# Crear una nueva tabla de rutas para la subnet privada
response_private_route_table = ec2.create_route_table(
    VpcId=vpc_id
)

private_route_table_id = response_private_route_table['RouteTable']['RouteTableId']
print(f'Tabla de rutas privada creada con ID: {private_route_table_id}')

# Crear una etiqueta para la tabla de rutas privada
ec2.create_tags(
    Resources=[private_route_table_id],
    Tags=[{'Key': 'Name', 'Value': 'PrivateRouteTable'}]
)

# Asociar la nueva tabla de rutas con la subnet privada
ec2.associate_route_table(
    RouteTableId=private_route_table_id,
    SubnetId=private_subnet_id
)

print('Configuración de VPC completa. La subnet pública tiene acceso a Internet y la subnet privada tiene su propia tabla de rutas.')

Subnet pública creada con ID: subnet-00cde9c01181ce876
Subnet privada creada con ID: subnet-0299cfec4a384f661
Internet Gateway creado con ID: igw-0ba5c6efc1677287f
Tabla de rutas predeterminada con ID: rtb-0aa614349600d8ab3
Tabla de rutas privada creada con ID: rtb-02f811a092171f3a4
Configuración de VPC completa. La subnet pública tiene acceso a Internet y la subnet privada tiene su propia tabla de rutas.


# Máquinas Virtuales (EC2 en AWS) 💻

Las máquinas virtuales son emulaciones de sistemas informáticos completos. En AWS, se implementan como instancias EC2 (Elastic Compute Cloud).

Características:
- Emula hardware completo 🖥️
- Control total sobre el sistema operativo ⚙️
- Requiere gestión y mantenimiento del sistema operativo 🛠️
- Escalado manual o automatizado, pero no instantáneo 📈
- Facturación por hora/segundo de ejecución ⏲️

## Ejemplo: Instancia EC2 AWS 🚀

En esta sección, se explica cómo lanzar una instancia EC2 en AWS, configurar su grupo de seguridad para permitir conexiones SSH usando EC2 Instance Connect y ejecutar un script de instalación de Docker en el momento del lanzamiento de la instancia.

### Creación y Configuración de un Grupo de Seguridad para EC2 Instance Connect 🚀

In [6]:
from botocore.exceptions import ClientError

# Especifica tu VPC y las IDs de subred
subnet_id = public_subnet_id

# Crea o obtiene un grupo de seguridad existente
security_group_name = 'EC2-InstanceConnect-SG'
try:
    security_group = ec2.create_security_group(
        GroupName=security_group_name,
        Description='Security group for EC2 Instance Connect',
        VpcId=vpc_id
    )
    
    # Agrega una regla de entrada para SSH desde el rango de IP de EC2 Instance Connect
    ec2.authorize_security_group_ingress(
        GroupId=security_group['GroupId'],
        IpPermissions=[
            {
                'IpProtocol': 'tcp',
                'FromPort': 22,
                'ToPort': 22,
                'IpRanges': [{'CidrIp': '18.206.107.24/29'}]  # Rango de IP de EC2 Instance Connect
            }
        ]
    )
    print("Grupo de seguridad creado exitosamente con regla de entrada SSH")
except ClientError as e:
    if e.response['Error']['Code'] == 'InvalidGroup.Duplicate':
        print("El grupo de seguridad ya existe. Usando el grupo existente.")
        security_groups = ec2.describe_security_groups(
            Filters=[
                {'Name': 'group-name', 'Values': [security_group_name]},
                {'Name': 'vpc-id', 'Values': [vpc_id]}
            ]
        )['SecurityGroups']
        if security_groups:
            security_group = security_groups[0]
        else:
            raise Exception(f"Grupo de seguridad {security_group_name} no encontrado en la VPC {vpc_id}")
    else:
        raise e


Grupo de seguridad creado exitosamente con regla de entrada SSH


### Lanza la instancia EC2 e instala docker 🚀

In [7]:
# Lee el contenido de docker_install.sh
with open('scripts/docker_install.sh', 'r') as file:
    docker_install_script = file.read()
    
try:
    user_data_script = f'''#!/bin/bash
        yum update -y
        yum install -y ec2-instance-connect
        echo "Instancia lanzada con EC2 Instance Connect instalado"

        # Script de instalación de Docker
        {docker_install_script}
    '''

    response = ec2.run_instances(
        ImageId='ami-04a81a99f5ec58529',  # AMI de Amazon Linux 2023
        InstanceType='t2.micro',
        MinCount=1,
        MaxCount=1,
        UserData=user_data_script,
        NetworkInterfaces=[{
            'SubnetId': subnet_id,
            'DeviceIndex': 0,
            'AssociatePublicIpAddress': True,
            'Groups': [security_group['GroupId']]
        }]
    )

    instance_id = response['Instances'][0]['InstanceId']
    print(f"Instancia EC2 creada con ID: {instance_id}")

    print("Configuración completa. La instancia se está lanzando.")
    print("Docker se instalará durante el lanzamiento de la instancia.")
    print("Deberías poder conectarte usando EC2 Instance Connect una vez que esté lista.")
except ClientError as e:
    print(f"Error al lanzar la instancia EC2: {e}")

Instancia EC2 creada con ID: i-069d1f9faca012a32
Configuración completa. La instancia se está lanzando.
Docker se instalará durante el lanzamiento de la instancia.
Deberías poder conectarte usando EC2 Instance Connect una vez que esté lista.


# Containers usando Docker y ECR 🐳🚀
En esta sección, exploramos cómo usar contenedores Docker y Amazon Elastic Container Service (ECS) para desplegar y gestionar aplicaciones en AWS.

**Docker:**
- 🐳 **Docker:** Una plataforma para desarrollar, enviar y ejecutar aplicaciones dentro de contenedores ligeros y portátiles.
- 📦 **Imágenes:** Archivos que contienen todo lo necesario para ejecutar una aplicación: código, runtime, bibliotecas y configuraciones.
- 🚀 **Contenedores:** Instancias ejecutables de imágenes Docker que pueden iniciarse y detenerse rápidamente.

**Amazon ECR:**
- 📦 **ECR (Elastic Container Registry):** Un servicio de almacenamiento completamente gestionado para imágenes Docker, que facilita el almacenamiento, la gestión y el despliegue de imágenes de contenedores.
- 🔒 **Seguridad:** Integración con IAM para controlar el acceso a los repositorios de imágenes.
- 🔄 **Automatización:** Soporte para pipelines de CI/CD para automatizar el despliegue de imágenes.


### Creación de repositorio ECR

In [8]:
import boto3
from botocore.exceptions import ClientError

# Crear un cliente para ECR
ecr_client = boto3.client('ecr')

# Nombre del repositorio ECR
repository_name = 'spend-wise-app'

try:
    # Crear el repositorio ECR
    response = ecr_client.create_repository(
        repositoryName=repository_name,
        imageTagMutability='MUTABLE',
        imageScanningConfiguration={'scanOnPush': True}
    )

    # Extraer y mostrar detalles del repositorio creado
    repository_uri = response['repository']['repositoryUri']
    print(f"Repositorio ECR creado exitosamente: {repository_uri}")

except ClientError as e:
    # Manejo de errores específicos de AWS
    if e.response['Error']['Code'] == 'RepositoryAlreadyExistsException':
        print(f"El repositorio {repository_name} ya existe.")
    else:
        print(f"Error al crear el repositorio ECR: {e}")


Repositorio ECR creado exitosamente: 021891591921.dkr.ecr.us-east-1.amazonaws.com/spend-wise-app


# Serverless (AWS Lambda) ⚡

La computación serverless permite ejecutar código sin provisionar ni administrar servidores. AWS Lambda es el servicio serverless de AWS.

Características:
- No hay que gestionar servidores 🛠️
- Escalado automático e instantáneo 📈
- Facturación por milisegundo de ejecución ⏱️
- Ideal para cargas de trabajo intermitentes 🌐
- Limitado en tiempo de ejecución y recursos ⏳

### Ejemplo: Crear una función Lambda

In [ ]:
def create_lambda_function():
    with open('lambda_function.zip', 'rb') as f:
        zipped_code = f.read()
    
    response = lambda_client.create_function(
        FunctionName='MyLambdaFunction',
        Runtime='python3.8',
        Role='arn:aws:iam::XXXXXXXXXXXX:role/lambda-role',  # IAM role ARN
        Handler='lambda_function.handler',
        Code=dict(ZipFile=zipped_code),
        Timeout=30
    )
    return response['FunctionArn']

lambda_arn = create_lambda_function()
print(f"Función Lambda creada con ARN: {lambda_arn}")

## Contenedores (ECS en AWS)

Los contenedores son unidades estándar de software que empaquetan el código y todas sus dependencias.
AWS ECS (Elastic Container Service) es un servicio de orquestación de contenedores.

Características:
- Ligeros y portables
- Aislamiento de aplicaciones
- Rápido despliegue y escalado
- Eficiente uso de recursos
- Ideal para microservicios

### Ejemplo: Crear un cluster ECS

In [ ]:
def create_ecs_cluster(cluster_name):
    response = ecs.create_cluster(clusterName=cluster_name)
    return response['cluster']['clusterArn']

cluster_arn = create_ecs_cluster('MiClusterECS')
print(f"Cluster ECS creado con ARN: {cluster_arn}")

## Comparación de servicios

1. IAM:
   - Gestión centralizada de accesos
   - Crucial para la seguridad en la nube
   - Se integra con todos los servicios AWS
   - Sin costo adicional

2. Máquinas Virtuales (EC2):
   - Mayor control y flexibilidad
   - Requiere más gestión
   - Mejor para aplicaciones que necesitan el sistema operativo completo
   - Costo: Puede ser más alto para cargas de trabajo intermitentes

3. Serverless (Lambda):
   - Sin gestión de servidores
   - Escalado automático
   - Ideal para microservicios y funciones específicas
   - Costo: Muy eficiente para cargas de trabajo variables

4. Contenedores (ECS):
   - Portabilidad y consistencia
   - Menos sobrecarga que las VMs
   - Bueno para microservicios y aplicaciones distribuidas
   - Costo: Equilibrio entre VMs y serverless

La elección depende de los requisitos específicos de tu aplicación, 
la experiencia de tu equipo y las necesidades de gestión y escalabilidad.